## Cleaning our scrapped Oscar's CSV ## 

In [1]:
import os

ruta_csv = os.path.abspath('nombre_del_archivo.csv')
print(ruta_csv)

/Users/davidusuariocasa/Desktop/Projects/ETL_project/nombre_del_archivo.csv


In [52]:
import pandas as pd


scrapped = pd.read_csv('/Users/davidusuariocasa/Desktop/Projects/ETL_project/Data/scrapped_not_clean.csv'
)


In [53]:
scrapped = scrapped.dropna() # First, we remove all columns that contain at least one null value

scrapped = scrapped.reset_index() # Now we reset the indexes

scrapped = scrapped.drop('index', axis=1) # And finally we remove the column 'index'


## Converting our 'duration' column to minutes ##

In [54]:
import re # We import the regex library

# We want to convert the values in our 'duration' column to minutes, so we create a function to do so

def convert_duration_to_minutes(value): 
    hours = re.findall(r'\d+h', value) # we use regex to get all values followed by an 'h' (hours) and store them
    minutes = re.findall(r'\d+min', value) # the same, but with mins

    hours = int(hours[0].rstrip('h')) if hours else 0 # If regex finds hours, we take out the 'h' and store the number
    minutes = int(minutes[0].rstrip('min')) if minutes else 0 # If regex finds mins, we take 'mins' and store the number

    total_minutes = hours * 60 + minutes # since we want everything in mins, we multiply the hours by 60 and add the remaining minutes

    return total_minutes # our function returns the total mins





In [55]:
scrapped['duration'] = scrapped['duration'].apply(convert_duration_to_minutes) # we apply the function to our 'duration' column


In [56]:
scrapped

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1927,1928,1,ACTOR,Emil Jannings,The Last Command,88,"8,0\n/10"
1,1927,1928,1,ART DIRECTION,William Cameron Menzies,The Dove;,90,"6,5\n/10"
2,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,Sunrise,94,"8,1\n/10"
3,1927,1928,1,CINEMATOGRAPHY,Karl Struss,Sunrise,94,"8,1\n/10"
4,1927,1928,1,DIRECTING (Comedy Picture),Lewis Milestone,Two Arabian Knights,92,"6,7\n/10"
...,...,...,...,...,...,...,...,...
1931,2022,2023,95,SOUND,"Mark Weingarten, James H. Mather, Al Nelson, C...",Top Gun: Maverick,130,"8,3\n/10"
1932,2022,2023,95,VISUAL EFFECTS,"Joe Letteri, Richard Baneham, Eric Saindon and...",Avatar: The Way of Water,192,"7,6\n/10"
1933,2022,2023,95,WRITING (Adapted Screenplay),Screenplay by Sarah Polley,Women Talking,104,"6,9\n/10"
1934,2022,2023,95,WRITING (Original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,139,"7,8\n/10"


## Changing our 'score' column ## 

In [57]:
def clean_rating(value):
    parts = value.split('\n/')
    if len(parts) == 2:
        try:
            cleaned_value = parts[0].replace(',', '.').strip()  # Reemplazar comas por puntos y eliminar espacios en blanco
            return float(cleaned_value)
        except ValueError:
            return None
    else:
        return None



In [58]:
scrapped['score'] = scrapped['score'].apply(clean_rating) # we apply the function to our 'score' column


In [59]:
scrapped

,year_film,year_ceremony,ceremony,category,name,film,duration,score
0,1927,1928,1,ACTOR,Emil Jannings,The Last Command,88,8.0
1,1927,1928,1,ART DIRECTION,William Cameron Menzies,The Dove;,90,6.5
2,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,Sunrise,94,8.1
3,1927,1928,1,CINEMATOGRAPHY,Karl Struss,Sunrise,94,8.1
4,1927,1928,1,DIRECTING (Comedy Picture),Lewis Milestone,Two Arabian Knights,92,6.7
...,...,...,...,...,...,...,...,...
1931,2022,2023,95,SOUND,"Mark Weingarten, James H. Mather, Al Nelson, C...",Top Gun: Maverick,130,8.3
1932,2022,2023,95,VISUAL EFFECTS,"Joe Letteri, Richard Baneham, Eric Saindon and...",Avatar: The Way of Water,192,7.6
1933,2022,2023,95,WRITING (Adapted Screenplay),Screenplay by Sarah Polley,Women Talking,104,6.9
1934,2022,2023,95,WRITING (Original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,139,7.8


## Changing our 'category' column ## 

In [60]:
# All the values of our 'category' column are in capital letters. We want them to be in lowercase, so we create a function

def capitalize_words(value):
    words = value.split()  # Divide the value in words
    capitalized_words = []  # List to store the capitalized words

    for word in words:
        # Capitalize the first letter of each word
        capitalized_word = word[0].upper() + word[1:].lower()
        capitalized_words.append(capitalized_word)  # Add the capitalized word to our list 
    # Now we join the capitalized words in a single string
    result = ' '.join(capitalized_words)
    return result



In [61]:
scrapped['category'] = scrapped['category'].apply(capitalize_words) # we apply the function to our 'category' column


## Changing our columns names ## 


In [63]:
# Finally we replace the values of my columns with the values from the 'new_names' list

new_names = ['Film year', 'Ceremony year', 'Ceremony number', 'Winning category', 'Winner name', 'Film title', 'Film Duration', 'Film score']

scrapped.columns = new_names


In [64]:
scrapped # This is our final dataframe

,Film year,Ceremony year,Ceremony number,Winning category,Winner name,Film title,Film Duration,Film score
0,1927,1928,1,Actor,Emil Jannings,The Last Command,88,8.0
1,1927,1928,1,Art Direction,William Cameron Menzies,The Dove;,90,6.5
2,1927,1928,1,Cinematography,Charles Rosher,Sunrise,94,8.1
3,1927,1928,1,Cinematography,Karl Struss,Sunrise,94,8.1
4,1927,1928,1,Directing (comedy Picture),Lewis Milestone,Two Arabian Knights,92,6.7
...,...,...,...,...,...,...,...,...
1931,2022,2023,95,Sound,"Mark Weingarten, James H. Mather, Al Nelson, C...",Top Gun: Maverick,130,8.3
1932,2022,2023,95,Visual Effects,"Joe Letteri, Richard Baneham, Eric Saindon and...",Avatar: The Way of Water,192,7.6
1933,2022,2023,95,Writing (adapted Screenplay),Screenplay by Sarah Polley,Women Talking,104,6.9
1934,2022,2023,95,Writing (original Screenplay),Written by Daniel Kwan & Daniel Scheinert,Everything Everywhere All at Once,139,7.8


In [65]:
scrapped.to_csv('scrapped_clean.csv', index=False) # We save our dataframe in CSV format